### Découpage par lesions 

In [ ]:
import os
import csv
import SimpleITK as sitk
import numpy as np

In [ ]:
# Dossier racine contenant tous les patients
BASE_DIR = "../../../27919209/MSLesSegDataset/train"  # à adapter selon votre structure

def classify(lesion_mean, ref_mean, ref_std, factor=1.5):
    """
    Classification basée sur l'écart-type (Z-score).
    [cite_start]L'amélioration utilise la dispersion naturelle du signal sain (ref_std)[cite: 2].
    Hypersignal : moyenne lésion > moyenne ref + (factor * écart-type ref)
    Hyposignal  : moyenne lésion < moyenne ref - (factor * écart-type ref)
    """
    upper_bound = ref_mean + (factor * ref_std)
    lower_bound = ref_mean - (factor * ref_std)
    
    if lesion_mean > upper_bound:
        return "hypersignal"
    elif lesion_mean < lower_bound:
        return "hyposignal"
    else:
        return "isosignal"

output_csv = "lesion_per_lesion_signal_stats_v2.csv"

with open(output_csv, mode="w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "patient",
        "session",
        "lesion_id",
        "modality",
        "lesion_mean",
        "lesion_min",
        "lesion_max",
        "ref_mean",
        "ref_std",
        "signal_class"
    ])

    # --- Boucle sur les patients ---
    for patient in sorted(os.listdir(BASE_DIR)):
        patient_dir = os.path.join(BASE_DIR, patient)
        if not os.path.isdir(patient_dir):
            continue

        # --- Boucle sur les sessions (T1, T2, etc.) ---
        for session in sorted(os.listdir(patient_dir)):
            session_dir = os.path.join(patient_dir, session)
            if not os.path.isdir(session_dir):
                continue

            prefix = f"{patient}_{session}"

            mask_path  = os.path.join(session_dir, f"{prefix}_MASK.nii.gz")
            t1_path    = os.path.join(session_dir, f"{prefix}_T1.nii.gz")
            t2_path    = os.path.join(session_dir, f"{prefix}_T2.nii.gz")
            flair_path = os.path.join(session_dir, f"{prefix}_FLAIR.nii.gz")

            if not (os.path.exists(mask_path)
                    and os.path.exists(t1_path)
                    and os.path.exists(t2_path)
                    and os.path.exists(flair_path)):
                print(f"[WARN] Fichiers manquants pour {patient} / {session}, on saute.")
                continue

            print(f"Traitement {patient} / {session} ...")

            # -- Lecture --
            mask_img_orig  = sitk.ReadImage(mask_path)
            t1_img_orig    = sitk.ReadImage(t1_path)
            t2_img_orig    = sitk.ReadImage(t2_path)
            flair_img_orig = sitk.ReadImage(flair_path)

            # Masque lésion binaire
            lesion_bin = mask_img_orig > 0

            # [cite_start]Lésions séparées par Connected Components pour obtenir des signatures individuelles [cite: 2]
            lesion_cc = sitk.ConnectedComponent(lesion_bin)

            # Masque "cerveau" simple (approximation par T1 > 0)
            brain_bin = t1_img_orig > 0

            # Référence = cerveau sain (cerveau moins toutes les zones de lésions)
            ref_bin   = sitk.And(brain_bin, sitk.Not(lesion_bin))
            ref_label = sitk.Cast(ref_bin, sitk.sitkUInt8)

            # Conversion en float64 pour les calculs statistiques
            modalities = {
                "T1": sitk.Cast(t1_img_orig, sitk.sitkFloat64),
                "T2": sitk.Cast(t2_img_orig, sitk.sitkFloat64),
                "FLAIR": sitk.Cast(flair_img_orig, sitk.sitkFloat64)
            }

            for modality_name, img in modalities.items():
                # Calcul des statistiques de référence (zone saine)
                stats_ref = sitk.LabelStatisticsImageFilter()
                stats_ref.Execute(img, ref_label)
                
                if not stats_ref.HasLabel(1):
                    continue
                
                ref_mean = stats_ref.GetMean(1)
                ref_std  = stats_ref.GetSigma(1) # Récupération de l'écart-type pour le point 1

                # Statistiques par lésion individuelle
                stats_lesions = sitk.LabelStatisticsImageFilter()
                stats_lesions.Execute(img, lesion_cc)

                for lesion_id in stats_lesions.GetLabels():
                    if lesion_id == 0:
                        continue  # Fond de l'image

                    lesion_mean = stats_lesions.GetMean(lesion_id)
                    lesion_min  = stats_lesions.GetMinimum(lesion_id)
                    lesion_max  = stats_lesions.GetMaximum(lesion_id)

                    # Utilisation de la nouvelle logique de classification
                    signal_class = classify(lesion_mean, ref_mean, ref_std)

                    writer.writerow([
                        patient,
                        session,
                        int(lesion_id),
                        modality_name,
                        round(float(lesion_mean), 4),
                        round(float(lesion_min), 4),
                        round(float(lesion_max), 4),
                        round(float(ref_mean), 4),
                        round(float(ref_std), 4),
                        signal_class
                    ])

print(f"Terminé. Résultats sauvegardés dans {output_csv}")

Traitement P1 / T1 ...
Traitement P1 / T2 ...
Traitement P1 / T3 ...
Traitement P10 / T1 ...
Traitement P10 / T2 ...
Traitement P11 / T1 ...
Traitement P11 / T2 ...
Traitement P12 / T1 ...
Traitement P12 / T2 ...
Traitement P12 / T3 ...
Traitement P12 / T4 ...
Traitement P13 / T1 ...
Traitement P13 / T2 ...
Traitement P14 / T1 ...
Traitement P14 / T2 ...
Traitement P14 / T3 ...
Traitement P14 / T4 ...
Traitement P15 / T1 ...
Traitement P16 / T1 ...
Traitement P17 / T1 ...
Traitement P18 / T1 ...
Traitement P19 / T1 ...
Traitement P19 / T2 ...
Traitement P19 / T3 ...
Traitement P19 / T4 ...
Traitement P2 / T1 ...
Traitement P2 / T2 ...
Traitement P2 / T3 ...
Traitement P2 / T4 ...
Traitement P20 / T1 ...
Traitement P20 / T2 ...
Traitement P20 / T3 ...
Traitement P21 / T1 ...
Traitement P22 / T1 ...
Traitement P22 / T2 ...
Traitement P23 / T1 ...
Traitement P24 / T1 ...
Traitement P25 / T1 ...
Traitement P26 / T1 ...
Traitement P27 / T1 ...
Traitement P28 / T1 ...
Traitement P28 / T2 ...